In [1]:
from IPython.display import HTML, Javascript
from NeuralNetwork import NeuralNetwork as NN
import json
import threading
import random
import MNIST
import numpy as np

In [2]:
normalize = np.vectorize(lambda x: np.float(x/255.0))

In [3]:
nn = NN(28*28,[32,16],10)
nn.learingRate = .1
answerkey = [0,1,2,3,4,5,6,7,8,9]
train_images =  MNIST.get_images('mnist/train-images-idx3-ubyte')
test_images  =  MNIST.get_images('mnist/t10k-images-idx3-ubyte')
train_labels =  MNIST.get_labels('mnist/train-labels-idx1-ubyte')
test_labels  =  MNIST.get_labels('mnist/t10k-labels-idx1-ubyte')

test_data = []
train_data = []

data_amount = len(train_images)
for i in range(data_amount):
    label=[0,0,0,0,0,0,0,0,0,0]
    label[train_labels[i]]=1
    train_data.append({'input':train_images[i],'target':label})
    
data_amount = len(test_images)   
for i in range(data_amount):
    label=[0,0,0,0,0,0,0,0,0,0]
    label[test_labels[i]]=1
    test_data.append({'input':test_images[i],'target':label})
    

In [4]:
def nnJson():
    print(nn.toJSON())

In [5]:
def train():
    random.shuffle(train_data)
    counter = 0;
    total = len(train_data)
    for data in train_data:
        nn.train(normalize(data['input']),data['target'])
        counter += 1
        print('{0:.2%} complete'.format(counter/total),end='\r')
    nn.predict(train_data[0]['input'])
        
    

In [6]:
def test():
    num_correct = 0
    counter = 0
    total = len(test_data)
    for data in test_data:
        prediction = nn.predict(normalize(data['input'])).argmax()
        if data['target'].index(1) == prediction:
            num_correct += 1
        counter += 1;
        print('{0:.2%} completed     '.format(counter/total),end='\r')
    score = num_correct / total
    print('{} out of {} correct'.format(num_correct,total))
    return score
    
    

In [7]:
accuracies = []

In [8]:
nn.learingRate = .1

In [16]:
counter = 0
#while accuracy < .89
while counter < 1:
    print('Testing')
    accuracy = test()
    print('accuracy:{0:.2%}'.format(accuracy))
    accuracies.append(accuracy)
    print('Training')
    train()
    counter += 1
    print('\nEpoch: {} Complete'.format(counter))


Testing
980 out of 10000 correct
accuracy:9.80%
Training
100.00% complete
Epoch: 1 Complete


In [12]:
print('accuracy:{0:.2%}'.format(test()))

980 out of 10000 correct
accuracy:9.80%


In [14]:
accuracies

[]

In [9]:
train_labels[111]

3

In [10]:
p = nn.predict(normalize(train_data[111]['input']))
p

array([[0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5]])

In [13]:
train_data[111]['target'].index(1) == p.argmax()

False

In [26]:
with open ('mnist_weights.json','w') as jfile:
    jfile.write(nn.toJSON())

In [31]:
with open ('mnist_weights.json','r') as jfile:
    data = json.loads(jfile.read())

In [ ]:
%%html
<h1>Neural Network Trainer</h1>


<div class="input-group">
  <div class="input-group-append">
    <span class="input-group-text">Cycles</span>
  </div>
  <input id="cycles" type="text" class="form-control" aria-label="Cycles" value='100'></input>
  <button id="go" class="btn btn-default">GO!</button>
</div>

<div>
    <div class='container'>
        <div class='row'>
          <div class='col-sm-4' id='weightCol'>
            Weights
          </div>
          <div class='col-sm-4' id='biasCol' style="width:150px;">
            Biases
          </div>
          <div class='col-sm-4' id='layerCol'style="width:150px;">
            Layers
          </div>
        </div>
        
    </div>
    
    
    <div id='errors'></div>
</div>

In [ ]:
%%javascript
function nnToTable(data) {
    //console.log(data)
    if(data.content.ename !== undefined ) {
        var content = data.content
        $('#errors').html(`<h1>${content.ename}</h1><h2>${content.evalue}</h2><p>${JSON.stringify(content.traceback)}</p>`);
        return
    } 
    
    var nndata = JSON.parse(data.content.text.trim());
    //console.log(nndata);
    var wtable = buildTableFromMatrix(nndata.weights[0])
    //console.log(`returned table:${wtable}`)
    //console.log(wtable)
     $('#weightCol').empty()
    $('#weightCol').append(`<h3>Weights</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#weightCol').append(buildTableFromMatrix(nndata.weights[i]))
    }
    $('#biasCol').empty()
    $('#biasCol').append(`<h3>Biases</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#biasCol').append(buildTableFromMatrix(nndata.biases[i]))
    }
    $('#layerCol').empty()
    $('#layerCol').append(`<h3>Layers</h3>`)
    for(var i = 0 ; i < nndata.weights.length; i++) {
        $('#layerCol').append(buildTableFromMatrix(nndata.layers[i]))
    }
    
    $('.row td').each(function() {
        var bg = toGreyscale(parseFloat($(this).text()));
        //console.log(gscale)
        var fg = 255-bg
        $(this).css('background-color',`rgb(${bg},${bg},${bg})`);
        $(this).css('color',`rgb(${fg},${fg},${fg})` )
    });
}


function buildTableFromMatrix(matrix) {
    //console.log(`Building Table from ${matrix.length} x ${matrix[0].length} matrix`)
    var table = $('<table>');
    table.addClass('table table-striped table-bordered table-dark table-sm')
    for(var row = 0; row < matrix.length; row++) {
        var tr = $('<tr>');
        for(var col = 0; col < matrix[row].length; col++ ) {
            tr.append(`<td>${matrix[row][col].toFixed(5)}</td>`)
        } table.append(tr)
    }
    
    return table
}

function map (num, in_min, in_max, out_min, out_max) {
  return (num - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;
}

function toGreyscale(num) {
    return map(num, 0.0,1.0,0,255)
}

var timer = null;
$('#go').click(function() {
    $('#go').attr('disabled','disabled');
     var cycles = 10
    if($.isNumeric($("#cycles").val())) {
      cycles = parseInt($("#cycles").val())
    }
    var kernel = IPython.notebook.kernel;
    var nnCallbacks = {
        iopub : {
            output: nnToTable,
            }
        };
    //kernel.execute('threading.thread(target=train, args=(10000,)).start()');
    var timerCount = 0
    timer = setInterval(function() {
        timerCount++
        //console.log(timerCount)
        $("#cycles").val(cycles-timerCount)
        kernel.execute('train(100)')
        if(timerCount > cycles) {
            $('#go').removeAttr('disabled');
            clearInterval(timer)
            console.log('suspending Timer')
        }
        kernel.execute('nnJson()',nnCallbacks)},100)
    
});